In [6]:
import pandas as pd

In [7]:
import mlflow

In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/150244868769783183', creation_time=1691158569332, experiment_id='150244868769783183', last_update_time=1691158569332, lifecycle_stage='active', name='StemToken', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/771411303543058092', creation_time=1691158216829, experiment_id='771411303543058092', last_update_time=1691158216829, lifecycle_stage='active', name='LemmaToken', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/761930417213375925', creation_time=1691151573324, experiment_id='761930417213375925', last_update_time=1691151573324, lifecycle_stage='active', name='RawToken', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1691118511104, experiment_id='0', last_update_time=1691118511104, lifecycle_stage='active', name='Default', tags={}>]

In [9]:
for exp in mlflow.search_experiments():
    print(exp.name, exp.experiment_id)

StemToken 150244868769783183
LemmaToken 771411303543058092
RawToken 761930417213375925
Default 0


In [17]:
for i in exp:
    print(i)

('artifact_location', 'mlflow-artifacts:/771411303543058092')
('creation_time', 1691158216829)
('experiment_id', '771411303543058092')
('last_update_time', 1691158216829)
('lifecycle_stage', 'active')
('name', 'LemmaToken')
('tags', {})


In [24]:
runs_df = mlflow.search_runs(experiment_ids=[761930417213375925])

In [26]:
runs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   run_id                         4 non-null      object             
 1   experiment_id                  4 non-null      object             
 2   status                         4 non-null      object             
 3   artifact_uri                   4 non-null      object             
 4   start_time                     4 non-null      datetime64[ns, UTC]
 5   end_time                       4 non-null      datetime64[ns, UTC]
 6   metrics.Test Accuracy          4 non-null      float64            
 7   metrics.F1 Score               4 non-null      float64            
 8   metrics.Training Accuracy      4 non-null      float64            
 9   params.n_estimators            4 non-null      object             
 10  params.Maximum Depth          

In [27]:
Run_ID = 'ec6ebf18026749ad8e54420838c50561'

# load from run id
logged_vect = f'runs:/{Run_ID}/vectorizer'
logged_model = f'runs:/{Run_ID}/model'
loaded_vect = mlflow.sklearn.load_model(logged_vect)
loaded_model = mlflow.pyfunc.load_model(logged_model)


In [100]:
run_id = str(loaded_model).split()[-1]
logged_vect = f'runs:/{run_id}/vectorizer'
loaded_vect = mlflow.sklearn.load_model(logged_vect)

In [101]:
loaded_vect

TfidfVectorizer()

In [ ]:
logged_vect = f'runs:/{Run_ID}/vectorizer'

In [29]:
loaded_vect

TfidfVectorizer()

In [31]:
import streamlit as st  
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text.lower())
    # filtered_words = [word for word in words if word.isalpha()]
    # filtered_words = [word for word in words if word.isalpha() and word not in stop_words]
    # filtered_words = [stemmer.stem(word) for word in words if word.isalpha() and word not in stop_words]
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(filtered_words)

In [46]:
text = "URGENT! You have won a 1 week FREE membership in our Â£100,000 Prize Jackpot!"

# get the run_id
Run_ID = 'ec6ebf18026749ad8e54420838c50561'

# load from run id
logged_vect = f'runs:/{Run_ID}/vectorizer'
logged_model = f'runs:/{Run_ID}/model'
loaded_vect = mlflow.sklearn.load_model(logged_vect)
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Preprocess the new raw text
processed_text = preprocess_text(text)

# Vectorize the processed text
vectorized_text = loaded_vect.transform([processed_text])

# Make prediction
prediction = loaded_model.predict(vectorized_text)

# Print the prediction
print(f"Predicted label: {prediction[0]}")

Predicted label: spam


In [84]:
loaded_model = mlflow.pyfunc.load_model(f"models:/spamfilter/Production")
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: ec6ebf18026749ad8e54420838c50561

In [89]:
x = (loaded_model.unwrap_python_model)

In [94]:
loaded_model.__doc__

"\n    MLflow 'python function' model.\n\n    Wrapper around model implementation and metadata. This class is not meant to be constructed\n    directly. Instead, instances of this class are constructed and returned from\n    :py:func:`load_model() <mlflow.pyfunc.load_model>`.\n\n    ``model_impl`` can be any Python object that implements the `Pyfunc interface\n    <https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#pyfunc-inference-api>`_, and is\n    returned by invoking the model's ``loader_module``.\n\n    ``model_meta`` contains model metadata loaded from the MLmodel file.\n    "

In [92]:
x()

MlflowException: Unable to retrieve base model object from pyfunc.

In [83]:
unwrapped_model = mlflow.unwrap_model(loaded_model)

AttributeError: module 'mlflow' has no attribute 'unwrap_model'

In [80]:
type(unwrapped_model)

method

In [68]:
mlflow.search_runs(run_view_type='Production')

ValueError: Unexpected view_type: Production

In [67]:
loaded_model.run_id

AttributeError: 'PyFuncModel' object has no attribute 'run_id'

In [ ]:
text = "URGENT! You have won a 1 week FREE membership in our Â£100,000 Prize Jackpot!"

# Specify the model name and version
model_name = "spamfilter"
model_version = "production"

# load from run id
logged_vect = f'runs:/{Run_ID}/vectorizer'
logged_model = f'runs:/{Run_ID}/model'
loaded_vect = mlflow.sklearn.load_model(logged_vect)
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")

# Preprocess the new raw text
processed_text = preprocess_text(text)

# Vectorize the processed text
vectorized_text = loaded_vect.transform([processed_text])

# Make prediction
prediction = loaded_model.predict(vectorized_text)

# Print the prediction
print(f"Predicted label: {prediction[0]}")

In [103]:
# This will fetch all registered models   
reg_models = mlflow.search_model_versions()

len(reg_models)

3

In [104]:
mlflow.search_model_versions()

[<ModelVersion: aliases=[], creation_timestamp=1691164375526, current_stage='Production', description='', last_updated_timestamp=1691169915423, name='spamfilter', run_id='ec6ebf18026749ad8e54420838c50561', run_link='', source='mlflow-artifacts:/771411303543058092/ec6ebf18026749ad8e54420838c50561/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>,
 <ModelVersion: aliases=[], creation_timestamp=1691164359798, current_stage='None', description='', last_updated_timestamp=1691164359798, name='spamfilter', run_id='314f48e5798b4ef281dfa78675be6d8f', run_link='', source='mlflow-artifacts:/150244868769783183/314f48e5798b4ef281dfa78675be6d8f/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>,
 <ModelVersion: aliases=[], creation_timestamp=1691164332732, current_stage='None', description='', last_updated_timestamp=1691164332732, name='spamfilter', run_id='4dff1536413043218338cec6f2cad8cb', run_link='', source='mlflow-artifact

In [62]:
reg_models[0]

<ModelVersion: aliases=[], creation_timestamp=1691164375526, current_stage='None', description='', last_updated_timestamp=1691164375526, name='spamfilter', run_id='ec6ebf18026749ad8e54420838c50561', run_link='', source='mlflow-artifacts:/771411303543058092/ec6ebf18026749ad8e54420838c50561/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [64]:
[[model.name, model.version, model.status, model.current_stage] for model in mlflow.search_model_versions()]

[['spamfilter', '3', 'READY', 'None'],
 ['spamfilter', '2', 'READY', 'None'],
 ['spamfilter', '1', 'READY', 'None']]

In [105]:
def get_production_model():
    try:        
        # This will fetch all registered models    
        prod = [model for model in mlflow.search_model_versions() if model.name == 'spamfilter' and model.current_stage == 'Production'] 
        model = prod[0]            
        print(f"Run_ID: {model.run_id}")
        print(f"Model Name: {model.name}")
        print(f"Version: {model.version}")               
    except:
        print("No Production Model Found")

get_production_model()

Run_ID: ec6ebf18026749ad8e54420838c50561
Model Name: spamfilter
Version: 3


In [119]:
# Specify the model name and version
model_name = "spamfilter"
model_version = "production"

# Load the production model
loaded_model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# Specify the artifact path for the vectorizer
vectorizer_artifact_path = "vectorizer"

# get run id by string operation
run_id = str(loaded_model).split()[-1]

# Load the vectorizer from the same run
loaded_vectorizer = mlflow.sklearn.load_model(model_uri=f"runs:/{run_id}/{vectorizer_artifact_path}")

In [120]:
text = "URGENT! You have won a 1 week FREE membership in our Â£100,000 Prize Jackpot!"
processed_text = preprocess_text(text)
vectorized_text = loaded_vect.transform([processed_text])
prediction = loaded_model.predict(vectorized_text)
print(f"Predicted label: {prediction[0]}")

Predicted label: spam


In [118]:
str(loaded_model).split()[-1]

'ec6ebf18026749ad8e54420838c50561'

In [115]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: ec6ebf18026749ad8e54420838c50561

In [122]:
def get_production_model():
    try:        
        # This will fetch all registered models    
        prod = [model for model in mlflow.search_model_versions() if model.name == 'spamfilter' and model.current_stage == 'SAtged'] 
        model = prod[0]            
        return model            
    except:
        print("No Production Model Found")

model = get_production_model()

No Production Model Found
